см readme
\\UR-JVV\Users\DAS\Desktop\GM TMP\Analytics\Проекты в работе\23-09-ХХ Переоценка ДК магазины выст\ReadMe2023-09-19.docx
20-09-2023 решено скачать весь КДМ с ДК 
Код 
- для подтягивания массы самородков в 
- подтягивания поля вставки из файла 01-01-2023 стандартную выгрузку из 1С



# imports

In [1]:
import numpy as np
import pandas as pd
import re
import os

In [2]:
from rusprice.dictdiamonds import dcol_dia, dcla_dia, dpo_dia, dogr_dia_discount
#import rusprice.func_rpdiamonds_xlsx_v2022_2_CARtoKVG_v2 as func_rpdiamonds_v2
#func_rpdiamonds_xlsx_v2023_1_CARtoKVG.py
import rusprice.func_rpdiamonds_xlsx_v2023_1_CARtoKVG as func_rpdiamonds_v2

In [3]:
folder = '//UR-JVV/Users/DAS/Desktop/GM TMP/Analytics/Проекты в работе/23-09-ХХ Переоценка ДК магазины выст/'
#cm readme

folder_res = folder+'for_script/'
folder_res

'//UR-JVV/Users/DAS/Desktop/GM TMP/Analytics/Проекты в работе/23-09-ХХ Переоценка ДК магазины выст/for_script/'

# Входные данные

## из 1С

In [4]:
#1. Вводные/

In [5]:
files = os.listdir(folder+'1 Вводные/')
files

['23-09-28 Выгрузка из ЮТД.xlsx',
 'Остатки1С__все_склады_ДК_2023-09-19_розница.xlsx',
 'Остатки1С_все_склады_ДК_2023-09-19_закупочные.xlsx',
 'Розница на 2023-09-19']

In [6]:
list(enumerate(files))

[(0, '23-09-28 Выгрузка из ЮТД.xlsx'),
 (1, 'Остатки1С__все_склады_ДК_2023-09-19_розница.xlsx'),
 (2, 'Остатки1С_все_склады_ДК_2023-09-19_закупочные.xlsx'),
 (3, 'Розница на 2023-09-19')]

In [15]:
filename_z = files[2]
filename_r= files[1]
filename_z, filename_r

('Остатки1С_все_склады_ДК_2023-09-19_закупочные.xlsx',
 'Остатки1С__все_склады_ДК_2023-09-19_розница.xlsx')

In [16]:
df_z = pd.read_excel(folder+'1 Вводные/'+filename_z,header=9)#, dtype={5: str}

In [17]:
#Вывести итог таблицы
df_z.iloc[len(df_z)-1][[ 'Количество конечный остаток', 'Конечный остаток вес',
       'Оценка (конечный остаток)']]

Количество конечный остаток         34528.0
Конечный остаток вес              123441.14
Оценка (конечный остаток)      888714246.96
Name: 34544, dtype: object

In [18]:
df_z.dropna(subset=['Код'], inplace=True)

In [19]:
#Вывести сумм таблицы
df_z[[ 'Количество конечный остаток', 'Конечный остаток вес',
       'Оценка (конечный остаток)']].sum()

Количество конечный остаток    3.452800e+04
Конечный остаток вес           1.234411e+05
Оценка (конечный остаток)      8.887142e+08
dtype: float64

In [20]:
# Загрузка файла с розничными ценами

In [21]:
df_r = pd.read_excel(folder+'1 Вводные/'+filename_r,header=9)#, dtype={5: str}

In [22]:
#Вывести итог таблицы
df_r.iloc[len(df_r)-1][[ 'Количество конечный остаток', 'Конечный остаток вес',
       'Оценка (конечный остаток)']]

Количество конечный остаток          34528.0
Конечный остаток вес               123441.14
Оценка (конечный остаток)      2935241509.89
Name: 34544, dtype: object

In [23]:
df_r.dropna(subset=['Код'], inplace=True)

In [24]:
#Вывести сумм таблицы
df_r[[ 'Количество конечный остаток', 'Конечный остаток вес',
       'Оценка (конечный остаток)']].sum()

Количество конечный остаток    3.452800e+04
Конечный остаток вес           1.234411e+05
Оценка (конечный остаток)      2.935242e+09
dtype: float64

In [25]:
df_r.rename(columns={'Оценка (конечный остаток)':'розничная стоимость'}, inplace=True)

In [26]:
df_z.rename(columns={'Оценка (конечный остаток)':'закупочная стоимость'}, inplace=True)

In [27]:
#проверка соответствия кодов
len(df_r['Код'].unique()) ,len(df_r)

(34528, 34528)

In [28]:
len(df_z['Код'].unique()) ,len(df_z)

(34528, 34528)

In [29]:
set(df_z['Код'].unique()) == set(df_r['Код'].unique())

True

In [30]:
#Присоединяем к стоимости df_1c
df_1c = pd.merge(df_r,df_z[['Код', 'закупочная стоимость']]
                           ,how='left',right_on='Код',left_on='Код')

In [31]:
df_1c[['закупочная стоимость','розничная стоимость']].sum()

закупочная стоимость    8.887142e+08
розничная стоимость     2.935242e+09
dtype: float64

In [32]:
df_1c.columns

Index(['Склад', 'Unnamed: 1', 'Unnamed: 2', 'Код', 'Unnamed: 4',
       'Старый код (Справочник "Номенклатура" (Общие))', 'Склад.1',
       'Склад.Склад / Магазин', 'Unnamed: 8',
       'Вставки (Справочник "Номенклатура" (Общие))', 'Вид номенклатуры',
       'Тип номенклатуры', 'Номенклатура', 'Группа',
       'Заводской номер (Справочник "Номенклатура" (Общие))',
       'Проба (Справочник "Номенклатура" (Общие))',
       'Цвет металла (Справочник "Номенклатура" (Общие))',
       'Дизайн (Справочник "Номенклатура" (Общие))', 'Артикул ',
       'Размерная группа Центрального камня (Справочник "Номенклатура" (Общие))',
       'Цвет Центрального Камня (Справочник "Номенклатура" (Общие))',
       'Чистота Центрального Камня (Справочник "Номенклатура" (Общие))',
       'Размер (Справочник "Номенклатура" (Общие))', 'Наименование для печати',
       'Единица измерения веса',
       'Вес Вставок ПДК (Карат) (Справочник "Номенклатура" (Общие))',
       'Кол-во ДК (Справочник "Номенклатура" (Об

In [33]:
for x in ['Склад', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 4','Unnamed: 8','Склад.1','Unnamed: 33',  'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31','Unnamed: 32']:
    print(x, df_1c[x].unique()[:13])

Склад [nan]
Unnamed: 1 [nan]
Unnamed: 2 [nan]
Unnamed: 4 [nan]
Unnamed: 8 [nan]
Склад.1 ['Благовещенск' 'Выставочный' 'Интернет-магазин' 'Иркутск' 'Кружало'
 'Ленина, 28' 'Ленина23' 'Ленск' 'Мирный' 'Московское представительство'
 'Новосибирск' 'Пассаж' 'Хабаровск']
Unnamed: 33 [nan 'Сахаювелир ООО']
Unnamed: 29 ['Ювелирные изделия' nan 'Драгоценности Якутии ООО' 'МИЗО'
 'Сахаювелир ООО' 'Кристалл 2001  ООО' 'СУВЕНИРЫ' 'Золото Якутии ОАО АПЮК'
 'Разные поставщики' '0']
Unnamed: 30 ['Саха Минералс Лимитед' 'Ювелирные изделия' 'ИП "Алексеев В.М."'
 '№ 06-329/2 от 12.12.16' 'Драгоценности Якутии ООО' 'Сахаювелир ООО'
 '№03/10-212 от 05.11.07' '№6/2-08-КП от 14.01.2008г.' 'Невский Бриллиант'
 'Золото Якутии ОАО АПЮК' 'Сапфир М ООО' '№11/08-КП от 21.01.2008г.'
 'Покровский гранильный завод ООО']
Unnamed: 31 ['Sakha Minerals Limited SM21-01 от 28.09.2021' 'Из Москвы'
 '10-271 от 21.12.2018' '06-329/2 от 12.12.2016 Задание №4'
 '06-341/4 от 26.12.2016' '03/10-95 от 15.05.08' '89/1/06 от 20.04

In [34]:
df_1c.drop(['Склад', 'Unnamed: 1', 'Unnamed: 2','Unnamed: 4','Unnamed: 8'], axis=1, inplace=True)

In [35]:
df_1c.rename(columns={'Склад.1':'Склад','Unnamed: 29':'родитель1_1c', 'Unnamed: 30':'родитель2_1c',
                      'Unnamed: 31':'родитель3_1c', 'Unnamed: 32':'родитель4_1c', 'Unnamed: 33':'родитель5_1c'},
            inplace=True)

In [36]:
df_1c

,Код,"Старый код (Справочник ""Номенклатура"" (Общие))",Склад,Склад.Склад / Магазин,"Вставки (Справочник ""Номенклатура"" (Общие))",Вид номенклатуры,Тип номенклатуры,Номенклатура,Группа,"Заводской номер (Справочник ""Номенклатура"" (Общие))",...,Группы,родитель1_1c,родитель2_1c,родитель3_1c,родитель4_1c,родитель5_1c,Количество конечный остаток,Конечный остаток вес,розничная стоимость,закупочная стоимость
0,ЦБ-00200503,NaN,Благовещенск,Розничный магазин,"45КР57-0,520-4/5А",Т ДМ Юи с ДК,Товар,Браслет,спецификация №1 от 27.01.2022г.,512,...,NaN,Ювелирные изделия,Саха Минералс Лимитед,Sakha Minerals Limited SM21-01 от 28.09.2021,спецификация №1 от 27.01.2022г.,NaN,1.0,7.89,185375.0,61791.55
1,ЦБ-00002854,МС00057368,Благовещенск,Розничный магазин,"2Кр57-0,083-2/2А 4Кр57-0,084-2/2А 6Кр57-0,280-...",Т ДМ Юи с ДК,Товар,Браслет,Товар ДК,"30575,51",...,NaN,NaN,Ювелирные изделия,Из Москвы,Товар ДК,NaN,1.0,13.10,333693.0,83173.87
2,ЦБ-00019951,МС00152100,Благовещенск,Розничный магазин,"1Кр57-0,086-4/5А",П ДМ Юи с ДК,Товар,Брошь,Броши,1191890,...,NaN,Ювелирные изделия,"ИП ""Алексеев В.М.""",10-271 от 21.12.2018,Броши,NaN,1.0,3.92,29041.0,9680.06
3,ЦБ-00019953,МС00146311,Благовещенск,Розничный магазин,"3Кр57-0,06-4/5А",П ДМ Юи с ДК,Товар,Брошь,накл.№5,1094,...,Ювелирные изделия,Драгоценности Якутии ООО,№ 06-329/2 от 12.12.16,06-329/2 от 12.12.2016 Задание №4,накл.№5,NaN,1.0,5.32,33789.0,14078.47
4,ЦБ-00111074,МС00111422,Благовещенск,Розничный магазин,"1Кр57-0,050-4/5А 1Кр57-0,031-6/6А 1Кр57-0,014-...",Т ДМ Юи с ДК,Товар,Брошь,51 ДК,422,...,NaN,Ювелирные изделия,Драгоценности Якутии ООО,06-341/4 от 26.12.2016,51 ДК,NaN,1.0,2.72,52866.0,21693.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34523,ЦБ-00182016,0000032928,Ювелирные изделия,Оптовый склад,NaN,Т ДМ Юи с ДК,Товар,серьги с ДК,"Акт №1 АК ""Золото Якутии""",01-12308,...,NaN,Ювелирные изделия,Золото Якутии ОАО АПЮК,Из магазина,"Акт №1 АК ""Золото Якутии""",NaN,1.0,3.71,NaN,5446.00
34524,ЦБ-00182017,0000032929,Ювелирные изделия,Оптовый склад,NaN,Т ДМ Юи с ДК,Товар,серьги с ДК,"Акт №1 АК ""Золото Якутии""",01-12309,...,NaN,Ювелирные изделия,Золото Якутии ОАО АПЮК,Из магазина,"Акт №1 АК ""Золото Якутии""",NaN,1.0,3.71,NaN,5446.00
34525,ЦБ-00182019,0000032934,Ювелирные изделия,Оптовый склад,NaN,Т ДМ Юи с ДК,Товар,серьги с ДК,"Акт №1 АК ""Золото Якутии""",01-14045,...,NaN,Ювелирные изделия,Золото Якутии ОАО АПЮК,Из магазина,"Акт №1 АК ""Золото Якутии""",NaN,1.0,3.67,NaN,3850.00
34526,ЦБ-00182020,0000032935,Ювелирные изделия,Оптовый склад,NaN,Т ДМ Юи с ДК,Товар,серьги с ДК,"Акт №1 АК ""Золото Якутии""",01-14046,...,NaN,Ювелирные изделия,Золото Якутии ОАО АПЮК,Из магазина,"Акт №1 АК ""Золото Якутии""",NaN,1.0,3.66,NaN,3850.00


In [37]:
#Если без самородков, пропуск 3
df_1c_res = df_1c

# Подтянуть массу самородков

## подтянуть масса самородков из '3.1. Самородки Сводка.xlsx',

In [33]:
files = os.listdir(folder+'1. Вводные/Веса самородков/')
files

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: '//UR-JVV/Users/DAS/Desktop/GM TMP/Analytics/Проекты в работе/23-09-ХХ Переоценка ДК магазины выст/1. Вводные/Веса самородков/'

In [ ]:
[x for x in enumerate(files)]

In [ ]:
df_sm1 = pd.read_excel(folder+'1. Вводные/Веса самородков/'+files[1])#,header=9)#, dtype={5: str}

In [ ]:
df_sm1.dropna(subset=['№ столбца'], inplace=True)

In [ ]:
 df_sm1[['Код', 'Артикул', 'Вес изделия','Вес самородка (общий суммированный)']]

In [ ]:
#проверка кода
len(df_sm1['Код'].unique()), len(df_sm1)

In [ ]:
#Присоединяем к df_1c
df_1c = pd.merge(df_1c ,df_sm1[['Код', 'Вес самородка (общий суммированный)']]
                           ,how='left',right_on='Код',left_on='Код')

In [ ]:
#присоеденил массу самородка к df_1c, проверяем сколько не получило массу 
df_1c['Вес самородка (общий суммированный)'].isna().sum()

In [ ]:
#Кто без цены
df_1c[df_1c['Вес самородка (общий суммированный)'].isna()]

In [ ]:
#df_1c['Проба самородка'].unique() -> array([894.,  nan, 885., 932., 889., 810., 890., 820., 875.])

## подтянуть масса самородков из '22-09-23 ДК и самородки На утверждение.xlsm'

### считываем файл срезаем самородки

In [ ]:
files = os.listdir(folder+'1. Вводные/Веса самородков/')
files

In [ ]:
[x for x in enumerate(files)]

In [ ]:
df_sm2 = pd.read_excel(folder+'1. Вводные/Веса самородков/'+files[0])#,header=9)#, dtype={5: str}

In [ ]:
df_sm2.columns

In [ ]:
df_sm2[['Наименование изделия', 'Артикул', 'Проба','Вес, грамм', 'Учётная стоимость','Вес золотого самородка (общий суммированный)']]

In [ ]:
df_sm2.dropna(subset=['Вес золотого самородка (общий суммированный)'], inplace=True)

### Создаем ключи и присоединяем массу самородка

In [ ]:
#проблема в том что данный файл без ключа, поэтому подтягиваем по совокупному совпадению - 
#- 'Номенклатура','Артикул ', 'Конечный остаток вес'

In [ ]:
#проверка уникальности ключа
len((df_sm2['Наименование изделия']+df_sm2['Артикул'].astype('str')+df_sm2['Вес, грамм'].astype('str')).unique())

In [ ]:
len(df_sm2)#.dropna(subset=['Вес, грамм']))

In [ ]:
df_1c_33 = df_1c[df_1c['Вес самородка (общий суммированный)'].isna()].copy()

In [ ]:
df_1c_33['key'] = df_1c_33[['Номенклатура','Артикул ',
  'Конечный остаток вес']].apply(lambda x: x['Номенклатура']+str(x['Артикул '])+str(x['Конечный остаток вес']),
                                 axis=1)

In [ ]:
df_sm2['key'] = df_sm2[['Наименование изделия', 'Артикул',
        'Вес, грамм']].apply(lambda x: x['Наименование изделия']+str(x['Артикул'])+str(x['Вес, грамм']),
                                 axis=1)

In [ ]:
len(set(df_sm2['key'])),len(df_sm2['key'])

In [ ]:
len(set(df_1c_33['key'])),len(df_1c_33['key'])

In [ ]:
#Этих из наличия нет в 27 утвержденных 9 шт из них  "Самородок Севера" понятно, тк это имитация  
set(df_1c_33['key']) - set(df_sm2['key'])

In [ ]:
#Эти есть в утверждении но нет в наличии - видимо проданы 
set(df_sm2['key'])-set(df_1c_33['key'])

In [ ]:
#set(df_1c_33['key']).intersection(set(df_sm2['key']))

In [ ]:
#Присоединяем к df_1c
df_1c_33 = pd.merge(df_1c_33,
         df_sm2[['Вес золотого самородка (общий суммированный)','key']],
         how='left',
         right_on=['key'],
         left_on=['key']
         )

In [ ]:
#Список без массы самородка
df_1c_33[df_1c_33['Вес золотого самородка (общий суммированный)'].isna()]

In [ ]:
df_1c_454 = df_1c[~df_1c['Вес самородка (общий суммированный)'].isna()].copy()

In [ ]:
folder_res

In [ ]:
#запись промрезультата для проверки на excel
pd.concat([df_1c_454,df_1c_33]).to_excel(folder_res+'df_1c_ms.xlsx')

## Сборка файла результата сборки весов самородков

In [ ]:
df_1c_454.columns

In [ ]:
df_1c_33.columns

In [ ]:
df_1c_33['вес самородка'] = df_1c_33['Вес золотого самородка (общий суммированный)']
df_1c_454['вес самородка'] = df_1c_454['Вес самородка (общий суммированный)']

In [ ]:
df_1c_res = pd.concat([df_1c_454,df_1c_33])

In [ ]:
df_1c_res['вес самородка'].isna().sum()

In [ ]:
#Вывести сумм таблицы
df_1c_res[[ 'Количество конечный остаток', 'Конечный остаток вес',
           'розничная стоимость', 'закупочная стоимость', 'вес самородка'
          ]].sum()

должно быть
Количество конечный остаток         487.00
Конечный остаток вес               3537.87
Оценка (конечный остаток)      12375195.96
dtype: float64

In [ ]:
df_1c_res['источник веса самородка'] = np.nan

In [ ]:
df_1c_res.loc[(~df_1c_res['Вес золотого самородка (общий суммированный)'].isna()),
              'источник веса самородка'] =  '22-09-23 ДК и самородки На утверждение.xlsm'

In [ ]:
df_1c_res.loc[(~df_1c_res['Вес самородка (общий суммированный)'].isna()),'источник веса самородка'] = '3.1. Самородки Сводка.xlsx'

In [ ]:
df_1c_res.columns

In [ ]:
df_1c_res = df_1c_res[['Код', 'Старый код (Справочник "Номенклатура" (Общие))', 'Склад',
       'Склад.Склад / Магазин', 'Вставки (Справочник "Номенклатура" (Общие))',
       'Вид номенклатуры', 'Тип номенклатуры', 'Номенклатура', 'Группа',
       'Заводской номер (Справочник "Номенклатура" (Общие))',
       'Проба (Справочник "Номенклатура" (Общие))',
       'Цвет металла (Справочник "Номенклатура" (Общие))',
       'Дизайн (Справочник "Номенклатура" (Общие))', 'Артикул ',
       'Размерная группа Центрального камня (Справочник "Номенклатура" (Общие))',
       'Цвет Центрального Камня (Справочник "Номенклатура" (Общие))',
       'Чистота Центрального Камня (Справочник "Номенклатура" (Общие))',
       'Размер (Справочник "Номенклатура" (Общие))', 'Наименование для печати',
       'Единица измерения веса',
       'Вес Вставок ПДК (Карат) (Справочник "Номенклатура" (Общие))',
       'Кол-во ДК (Справочник "Номенклатура" (Общие))',
       'Вес Вставок ДК (Карат) (Справочник "Номенклатура" (Общие))', 'Группы',
       'родитель1', 'родитель2', 'родитель3', 'родитель4',
       'Количество конечный остаток', 'Конечный остаток вес',
       'розничная стоимость', 'закупочная стоимость',
       'вес самородка',
       'источник веса самородка']]

# Подтянуть вставки ШК, родитель 1 -5

o	Данные по вставкам надежнее чем с 1с (на 1 янв 2023г) \\UR-JVV\Users\DAS\Desktop\GM TMP\Analytics\Проекты в работе\23-09-XX Переоценка Самородки\1. Вводные\Данные по вставкам\23-01-01 Сводка ЮИ.xlsx

In [38]:
files = os.listdir('//UR-JVV/Users/DAS/Desktop/GM TMP/Analytics/Проекты в работе/23-09-XX Переоценка Самородки/1. Вводные/Данные по вставкам/')
files

['2.4 Сводка Описания вставок V2.xlsx', '23-01-01 Сводка ЮИ.xlsx']

In [39]:
list(enumerate(files))

[(0, '2.4 Сводка Описания вставок V2.xlsx'), (1, '23-01-01 Сводка ЮИ.xlsx')]

In [40]:
fn = files[1]
fn
# 1, '23-01-01 Сводка ЮИ.xlsx' основной файл на 01-01-2023, 
# 0  '2.4 Сводка Описания вставок V2.xlsx' - неисправный файл был отклонен

'23-01-01 Сводка ЮИ.xlsx'

In [41]:
df_vst = pd.read_excel('//UR-JVV/Users/DAS/Desktop/GM TMP/Analytics/Проекты в работе/23-09-XX Переоценка Самородки/1. Вводные/Данные по вставкам/'\
                       +fn)#,header=9)#, dtype={5: str}

In [42]:
df_vst.columns

Index(['Source.Name', 'Штрихкод', 'Старый код', 'Код', 'Наименование',
       'Артикул', 'Зав Ном', 'Вид ном', 'Проба', 'Цвет металла', 'Кол', 'Вес',
       'Закуп', 'Розн', 'Вставки', 'Размер', 'РазмернаяГруппаЦК', 'ЦветЦК',
       'ЧистотаЦК', 'Дизайн', 'Родитель1', 'Родитель2', 'Родитель3',
       'Родитель4', 'Родитель5'],
      dtype='object')

In [43]:
len(df_vst['Код'].unique()), len(df_vst)

(143861, 143861)

In [44]:
len(df_1c_res['Код'].unique()), len(df_1c_res)

(34528, 34528)

In [45]:
#Присоединяем к df_1c
df_1c_res = pd.merge(df_1c_res,
         df_vst[['Код','Вставки', 'Штрихкод', 'Старый код',
                 'Родитель1', 'Родитель2', 'Родитель3',
                 'Родитель4', 'Родитель5' ]],
         how='left',
         right_on=['Код'],
         left_on=['Код']
         )

In [46]:
#223 пустых строк это по 3 договорам новые Валив и ДЯ
df_1c_res['Штрихкод'].isna().sum()

223

# Новый файл для вставок (не используется )

In [68]:
files = os.listdir('//UR-JVV/Users/DAS/Desktop/GM TMP/Analytics/Проекты в работе/23-09-XX Переоценка Самородки/1. Вводные/Данные по вставкам/')
list(enumerate(files))

[(0, '2.4 Сводка Описания вставок V2.xlsx'), (1, '23-01-01 Сводка ЮИ.xlsx')]

In [70]:
df_vst2 = pd.read_excel('//UR-JVV/Users/DAS/Desktop/GM TMP/Analytics/Проекты в работе/23-09-XX Переоценка Самородки/1. Вводные/Данные по вставкам/'+files[0])#,header=9)#, dtype={5: str}

In [71]:
df_vst2.columns

Index(['Склад', 'Штрихкод', 'Старый код', 'Код', 'Наименование', 'Артикул',
       'Зав Ном', 'Вид ном', 'Проба', 'Цвет металла', 'Кол', 'Вес', 'Закуп',
       'Розн', 'Размер', 'РазмернаяГруппаЦК', 'ЦветЦК', 'ЧистотаЦК', 'Дизайн',
       'Родитель1', 'Родитель2', 'Родитель3', 'Родитель4', 'Родитель5',
       'Код классификатора Гохран', 'Масса, гр. В хим чистоте', 'Кол-во ДК',
       'Вес вставок ДК(карат)', 'Вес вставок ПДК (Карат)', 'Сводка Вставки',
       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32'],
      dtype='object')

In [73]:
df_vst2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153278 entries, 0 to 153277
Data columns (total 33 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Склад                      153278 non-null  object 
 1   Штрихкод                   153170 non-null  float64
 2   Старый код                 137784 non-null  object 
 3   Код                        153270 non-null  object 
 4   Наименование               153270 non-null  object 
 5   Артикул                    143703 non-null  object 
 6   Зав Ном                    70188 non-null   object 
 7   Вид ном                    153270 non-null  object 
 8   Проба                      143875 non-null  float64
 9   Цвет металла               153153 non-null  object 
 10  Кол                        153270 non-null  float64
 11  Вес                        149778 non-null  float64
 12  Закуп                      153270 non-null  float64
 13  Розн                       11

In [75]:
len(df_vst2['Код'].unique()), len(df_vst2)

(152672, 153278)

In [ ]:
  ['Т ДМ Юи с ДК', 'Т ДМ Юи сер. с ДК','П ДМ Юи с ДК','Т ДМ Юи Каст ДК','П ДМ Юи Каст ДК']

In [81]:
(len(df_vst2.loc[(df_vst2['Вид ном'].isin( ['Т ДМ Юи с ДК', 'Т ДМ Юи сер. с ДК','П ДМ Юи с ДК',
                                                   'Т ДМ Юи Каст ДК','П ДМ Юи Каст ДК'])),'Код'].unique()),
                 len(df_vst2.loc[(df_vst2['Вид ном'].isin(  ['Т ДМ Юи с ДК', 'Т ДМ Юи сер. с ДК','П ДМ Юи с ДК',
                                                             'Т ДМ Юи Каст ДК','П ДМ Юи Каст ДК'])),'Код']))

(36398, 36398)

In [84]:
df_vst2_DK = df_vst2.loc[(df_vst2['Вид ном'].isin(['Т ДМ Юи с ДК', 'Т ДМ Юи сер. с ДК','П ДМ Юи с ДК',
                                                             'Т ДМ Юи Каст ДК','П ДМ Юи Каст ДК']))]

In [88]:
df_vst2_DK.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36398 entries, 1758 to 153277
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Склад                      36398 non-null  object 
 1   Штрихкод                   36388 non-null  float64
 2   Старый код                 30683 non-null  object 
 3   Код                        36398 non-null  object 
 4   Наименование               36398 non-null  object 
 5   Артикул                    36397 non-null  object 
 6   Зав Ном                    35861 non-null  object 
 7   Вид ном                    36398 non-null  object 
 8   Проба                      36398 non-null  float64
 9   Цвет металла               36304 non-null  object 
 10  Кол                        36398 non-null  float64
 11  Вес                        36398 non-null  float64
 12  Закуп                      36398 non-null  float64
 13  Розн                       21664 non-null 

In [40]:
len(df_1c_res['Код'].unique()), len(df_1c_res)

(34528, 34528)

In [87]:
len(set(df_1c_res['Код']) - set(df_vst2_DK['Код']))

214

In [89]:
36398-34528

1870

In [90]:
#Присоединяем к df_1c
df_1c_res = pd.merge(df_1c_res,
         df_vst2_DK[['Код','Сводка Вставки']],
         how='left',
         right_on=['Код'],
         left_on=['Код']
         )

In [91]:
df_1c_res['Штрихкод'].isna().sum()

223

In [92]:
df_1c_res['Сводка Вставки'].isna().sum()

214

# Запись результата в файл (не используется) 

In [93]:
folder_res

'//UR-JVV/Users/DAS/Desktop/GM TMP/Analytics/Проекты в работе/23-09-ХХ Переоценка ДК магазины выст/for_script/'

In [44]:
#запись промрезультата для проверки на excel
#df_1c_res.to_excel(folder_res+'Остатки1С__все_склады_ДК_2023-09-19.xlsx')

# Анализ 0

In [40]:
df_1c_res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34528 entries, 0 to 34527
Data columns (total 41 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   Код                                                                      34528 non-null  object 
 1   Старый код (Справочник "Номенклатура" (Общие))                           28415 non-null  object 
 2   Склад                                                                    34528 non-null  object 
 3   Склад.Склад / Магазин                                                    34528 non-null  object 
 4   Вставки (Справочник "Номенклатура" (Общие))                              24495 non-null  object 
 5   Вид номенклатуры                                                         34528 non-null  object 
 6   Тип номенклатуры                                                      

In [41]:
df_1c_res.columns

Index(['Код', 'Старый код (Справочник "Номенклатура" (Общие))', 'Склад',
       'Склад.Склад / Магазин', 'Вставки (Справочник "Номенклатура" (Общие))',
       'Вид номенклатуры', 'Тип номенклатуры', 'Номенклатура', 'Группа',
       'Заводской номер (Справочник "Номенклатура" (Общие))',
       'Проба (Справочник "Номенклатура" (Общие))',
       'Цвет металла (Справочник "Номенклатура" (Общие))',
       'Дизайн (Справочник "Номенклатура" (Общие))', 'Артикул ',
       'Размерная группа Центрального камня (Справочник "Номенклатура" (Общие))',
       'Цвет Центрального Камня (Справочник "Номенклатура" (Общие))',
       'Чистота Центрального Камня (Справочник "Номенклатура" (Общие))',
       'Размер (Справочник "Номенклатура" (Общие))', 'Наименование для печати',
       'Единица измерения веса',
       'Вес Вставок ПДК (Карат) (Справочник "Номенклатура" (Общие))',
       'Кол-во ДК (Справочник "Номенклатура" (Общие))',
       'Вес Вставок ДК (Карат) (Справочник "Номенклатура" (Общие))', 'Гру

In [42]:
for c in ['Родитель1', 'Родитель2', 'Родитель3', 'Родитель4', 'Родитель5']:
    print(c, '-->', df_1c_res.sample(10)[c])

Родитель1 --> 9609     Ювелирные изделия
15919    Ювелирные изделия
9506     Ювелирные изделия
25696    Ювелирные изделия
27876    Ювелирные изделия
19097    Ювелирные изделия
9740     Ювелирные изделия
20744    Ювелирные изделия
11052    Ювелирные изделия
7976     Ювелирные изделия
Name: Родитель1, dtype: object
Родитель2 --> 24214    Драгоценности Якутии ООО
4386        Саха Минералс Лимитед
25781      Золото Якутии ОАО АПЮК
34433                         NaN
24971    Драгоценности Якутии ООО
14771          ИП "Алексеев В.М."
15624          ИП "Алексеев В.М."
25337    Драгоценности Якутии ООО
9783           Кристалл 2001  ООО
31886          Кристалл 2001  ООО
Name: Родитель2, dtype: object
Родитель3 --> 11176                  07/10-587-09 от 11.12.09
4479     №10-121 от 29.03.12 Доп.10 от 29.03.13
27529                №6/2-08-КП от 14.01.2008г.
19361                №6/2-08-КП от 14.01.2008г.
18721                №1/1-08-КП от 09.01.2008г.
16405       06-140 от 30.03.2012 доп.9 12.08.1

In [43]:
for c in ['родитель1_1c', 'родитель2_1c', 'родитель3_1c', 'родитель4_1c', 'родитель5_1c']:
    print(c, '-->\n', df_1c_res.sample(10)[c])

родитель1_1c -->
 28307          Кристалл 2001  ООО
15889           Ювелирные изделия
30558           Ювелирные изделия
34196           Ювелирные изделия
1866            Ювелирные изделия
24039    Драгоценности Якутии ООО
6702            Ювелирные изделия
31189           Ювелирные изделия
16611           Ювелирные изделия
27408    Драгоценности Якутии ООО
Name: родитель1_1c, dtype: object
родитель2_1c -->
 19205                №6/2-08-КП от 14.01.2008г.
25674                    Золото Якутии ОАО АПЮК
18609                  Драгоценности Якутии ООО
6244                     №03/10-212 от 05.11.07
17256                            Сахаювелир ООО
22965    Договор комиссии №03/10-31 от 28.02.02
3288                      Саха Минералс Лимитед
10678                  Драгоценности Якутии ООО
15307                    № 06-329/2 от 12.12.16
10792                     Акт №1 от 21.10.05 г.
Name: родитель2_1c, dtype: object
родитель3_1c -->
 21094                    48/03-2019 от 26.09.19
3751      

In [44]:
df_1c_res.value_counts('Склад')

Склад
Ювелирные изделия               16505
Центральный                      1793
Иркутск                          1772
Ленина23                         1750
Пассаж                           1619
Ленина, 28                       1584
Кружало                          1564
Новосибирск                      1413
Хабаровск                        1343
Ленск                            1253
Мирный                           1156
Благовещенск                     1046
Интернет-магазин                  861
Выставочный                       818
Московское представительство       51
dtype: int64

In [45]:
#Где находятся 223 с не подключенным штрихкодом
df_1c_res[df_1c_res['Штрихкод'].isna()].value_counts('Склад')

Склад
Ювелирные изделия               171
Московское представительство     43
Кружало                           3
Центральный                       3
Ленина, 28                        2
Пассаж                            1
dtype: int64

In [46]:
#Где находятся 214 с не подключенным Сводка Вставки
df_1c_res[df_1c_res['Сводка Вставки'].isna()].value_counts('Склад')

KeyError: 'Сводка Вставки'

In [47]:
#Где находятся 213 с не подключенным полем вставки
df_1c_res[df_1c_res['Вставки'].isna()].value_counts('Склад')

Склад
Ювелирные изделия               171
Московское представительство     33
Кружало                           3
Центральный                       3
Ленина, 28                        2
Пассаж                            1
dtype: int64

In [48]:
#По каким договорам 223 с не подключенным полем Вставки
df_1c_res[df_1c_res['Вставки'].isna()].value_counts(['родитель2_1c','родитель3_1c','родитель4_1c'])

родитель2_1c              родитель3_1c                родитель4_1c       
Драгоценности Якутии ООО  №10-171 от 13.10.2022       РТ-5 от 24.10.2022     113
                          №103/03-2022 от 27.12.2022  Рт-25 от 28.12.2022     66
ОСОО "Валив"              №V22-01 от 07.07.2022       эксклюзив               34
dtype: int64

In [49]:
#По каким договорам 223 с не подключенным полем штрихкод
df_1c_res[df_1c_res['Штрихкод'].isna()].value_counts(['родитель2_1c','родитель3_1c','родитель4_1c'])

родитель2_1c              родитель3_1c                родитель4_1c        
Драгоценности Якутии ООО  №10-171 от 13.10.2022       РТ-5 от 24.10.2022      113
                          №103/03-2022 от 27.12.2022  Рт-25 от 28.12.2022      66
ОСОО "Валив"              №V22-01 от 07.07.2022       эксклюзив                34
Ювелирные изделия         ИП "Алексеев В.М."          10-064 от 07.04.2021     10
dtype: int64

In [99]:
#По каким договорам 213 с не подключенным полем Сводка Вставки
df_1c_res[df_1c_res['Сводка Вставки'].isna()].value_counts(['родитель2_1c','родитель3_1c','родитель4_1c'])

родитель2_1c              родитель3_1c                          родитель4_1c                            
Драгоценности Якутии ООО  №10-171 от 13.10.2022                 РТ-5 от 24.10.2022                          113
                          №103/03-2022 от 27.12.2022            Рт-25 от 28.12.2022                          66
ОСОО "Валив"              №V22-01 от 07.07.2022                 эксклюзив                                    34
№03/10-212 от 05.11.07    ДЯ 03/10-212 от 05.11.07 (15 партия)  Рнк-696/290 от 09.11.07г. ДК (15 партия)      1
dtype: int64

In [100]:
df_1c_res[df_1c_res['Сводка Вставки'].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 5449 to 34458
Data columns (total 42 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   Код                                                                      214 non-null    object 
 1   Старый код (Справочник "Номенклатура" (Общие))                           1 non-null      object 
 2   Склад                                                                    214 non-null    object 
 3   Склад.Склад / Магазин                                                    214 non-null    object 
 4   Вставки (Справочник "Номенклатура" (Общие))                              214 non-null    object 
 5   Вид номенклатуры                                                         214 non-null    object 
 6   Тип номенклатуры                                                     

In [101]:
#df_1c_res[df_1c_res['Сводка Вставки'].isna()]

,Код,"Старый код (Справочник ""Номенклатура"" (Общие))",Склад,Склад.Склад / Магазин,"Вставки (Справочник ""Номенклатура"" (Общие))",Вид номенклатуры,Тип номенклатуры,Номенклатура,Группа,"Заводской номер (Справочник ""Номенклатура"" (Общие))",...,закупочная стоимость,Вставки,Штрихкод,Старый код,Родитель1,Родитель2,Родитель3,Родитель4,Родитель5,Сводка Вставки
5449,ЦБ-00210173,NaN,Кружало,Розничный магазин,"4КР17-0,011-2/3А",Т ДМ Юи с ДК,Товар,Подвеска,Рт-25 от 28.12.2022,6432200344300272,...,18054.46,"4КР17-0,011-2/3А",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5999,ЦБ-00210155,NaN,Кружало,Розничный магазин,"1КР57-0,120-4/4А 1КР57-0,120-4/4А",Т ДМ Юи с ДК,Товар,Серьги,Рт-25 от 28.12.2022,6432205314748483,...,56645.87,"1КР57-0,120-4/4А 1КР57-0,120-4/4А",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6000,ЦБ-00210166,NaN,Кружало,Розничный магазин,"1КР57-0,140-4/4А 1КР57-0,140-4/4А",Т ДМ Юи с ДК,Товар,Серьги,Рт-25 от 28.12.2022,6432205331751500,...,60708.12,"1КР57-0,140-4/4А 1КР57-0,140-4/4А",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7102,ЦБ-00210175,NaN,"Ленина, 28",Розничный магазин,"4КР17-0,011-2/3А",Т ДМ Юи с ДК,Товар,Подвеска,Рт-25 от 28.12.2022,6432200344300300,...,18731.50,"4КР17-0,011-2/3А",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7639,ЦБ-00210157,NaN,"Ленина, 28",Розничный магазин,"1КР57-0,120-4/4А 1КР57-0,120-4/4А",Т ДМ Юи с ДК,Товар,Серьги,Рт-25 от 28.12.2022,6432205314736628,...,57097.23,"1КР57-0,120-4/4А 1КР57-0,120-4/4А",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34454,ЦБ-00212005,NaN,Ювелирные изделия,Оптовый склад,"1КР57-0,050-3/3А 1КР57-0,050-3/3А",Т ДМ Юи с ДК,Товар,Серьги,РТ-5 от 24.10.2022,6432200344293237,...,37873.35,"1КР57-0,050-3/3А 1КР57-0,050-3/3А",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34455,ЦБ-00212011,NaN,Ювелирные изделия,Оптовый склад,"1КР57-0,016-4/5А 1КР57-0,016-4/5А",Т ДМ Юи с ДК,Товар,Серьги,РТ-5 от 24.10.2022,6432202511709350,...,14683.10,"1КР57-0,016-4/5А 1КР57-0,016-4/5А",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34456,ЦБ-00212012,NaN,Ювелирные изделия,Оптовый склад,"1КР57-0,016-4/5А 1КР57-0,016-4/5А",Т ДМ Юи с ДК,Товар,Серьги,РТ-5 от 24.10.2022,6432202511709374,...,14683.10,"1КР57-0,016-4/5А 1КР57-0,016-4/5А",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34457,ЦБ-00212013,NaN,Ювелирные изделия,Оптовый склад,"1КР57-0,016-4/5А 1КР57-0,016-4/5А",Т ДМ Юи с ДК,Товар,Серьги,РТ-5 от 24.10.2022,6432202511709396,...,14531.50,"1КР57-0,016-4/5А 1КР57-0,016-4/5А",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
#Проверка на срезанность вставок по 100 символов
df_1c_res.loc[df_1c_res['Вставки'].isna(),'Вставки (Справочник "Номенклатура" (Общие))'].map(len).unique()

array([ 16,  33,  35, 103, 127, 105,  37,  34,  87,  70,  52,  54,  73,
        51,  69, 107, 122,  85, 104,  71, 120, 137,  36,  55,  17,  31,
       119,  24], dtype=int64)

# Подтянуть недостающие вставки из вставок 1С

In [51]:
df_1c_res.loc[df_1c_res['Вставки'].isna(),'Вставки'] = df_1c_res['Вставки (Справочник "Номенклатура" (Общие))']

In [52]:
df_1c_res.loc[df_1c_res['Вставки'].isna(),'Вставки']

Series([], Name: Вставки, dtype: object)

In [53]:
df_1c_res[['Вставки']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34528 entries, 0 to 34527
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Вставки  34528 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


# Подтянуть недостающие Поставщик, договор, накладная из 1C

In [60]:
list_rod_rod_1c = [('Родитель2','родитель2_1c'),
 ('Родитель3','родитель3_1c'),
 ('Родитель4','родитель4_1c'),
 ('Родитель5','родитель5_1c')]
 

In [62]:
for rod,rod_1c in list_rod_rod_1c:
   df_1c_res.loc[df_1c_res[rod].isna(),rod] = df_1c_res.loc[df_1c_res[rod].isna(),rod_1c]

In [57]:
df_1c_res.loc[df_1c_res['Родитель2'].isna(),'Родитель2'] = df_1c_res.loc[df_1c_res['Родитель2'].isna(),'родитель2_1c']

In [58]:
df_1c_res.loc[df_1c_res['Родитель2'].isna(),'Родитель2']

Series([], Name: Родитель2, dtype: object)

In [59]:
df_1c_res[['Родитель2']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34528 entries, 0 to 34527
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Родитель2  34528 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


# Запись результата в файл 2

In [63]:
folder_res

'//UR-JVV/Users/DAS/Desktop/GM TMP/Analytics/Проекты в работе/23-09-ХХ Переоценка ДК магазины выст/for_script/'

In [64]:
#запись промрезультата для проверки на excel
df_1c_res.to_excel(folder_res+'Остатки1С__все_склады_ДК_2023-09-25_13-00.xlsx')

In [109]:
#df_1c_res.loc[df_1c_res['Код'] =='ЦБ-00040270','Сводка Вставки' ].to_list()

['1Г56-1,15-5/4Б 1Ов57-0,51-8-5/7а 1Ов57-0,-8.5/4А 10кр57-0,14-3/5А 36кр57-0,35-3/5А 147кр57-0,83-3/5А']

# del

In [ ]:
'Родитель1', 'Родитель2', 'Родитель3', 'Родитель4', 'Родитель5'

o	Данные по вставкам надежнее чем с 1с (на 1 янв 2023г) 1. Вводные\Данные по вставкам\23-01-01 Сводка ЮИ.xlsx

In [ ]:
files = os.listdir(folder+'1. Вводные/Данные по вставкам/')
files

[x for x in enumerate(files)]

df_vst = pd.read_excel(folder+'1. Вводные/Данные по вставкам/'+files[0])#,header=9)#, dtype={5: str}

df_vst.columns

len(df_vst['Код'].unique()), len(df_vst)

len(df_1c_res['Код'].unique()), len(df_1c_res)

#Присоединяем к df_1c
df_1c_res = pd.merge(df_1c_res,
         df_vst[['Код','Вставки', 'Штрихкод', 'Старый код',
                 'Родитель1', 'Родитель2', 'Родитель3',
                 'Родитель4', 'Родитель5' ]],
         how='left',
         right_on=['Код'],
         left_on=['Код']
         )

df_1c_res['Штрихкод'].isna().sum()